In [1]:
import pandas as pd
from azureml.core import Workspace, Dataset

ws = Workspace.from_config()

In [2]:
data = Dataset.get_by_name(ws, 'sample_features_galliera').to_pandas_dataframe()
X = data

In [13]:
scoring_sample = X.sample(10)

In [14]:
scoring_sample

,RAC3_tot,RAC3_ok,RAC3_low,RAC3_hight,RAC3_trend,TRIG_tot,TRIG_ok,TRIG_low,TRIG_hight,TRIG_problem,...,perc_ricoverato,num_prestazioni_tot,n_ricoveri,n_days_tot_rec,max_days_rec,mean_days_rec,sesso_F,sesso_M,decesso_False,decesso_True
5035,1,1.0,0.0,0.0,0.0,1,1.0,0.0,0.0,0.0,...,0.103960,202,0,0,0,0.000000,0,1,0,1
4220,0,0.0,0.0,0.0,-99.0,1,1.0,0.0,0.0,0.0,...,0.142857,14,1,1,1,1.000000,0,1,1,0
8471,0,0.0,0.0,0.0,-99.0,1,1.0,0.0,0.0,0.0,...,0.444444,36,0,0,0,0.000000,0,1,0,1
2450,0,0.0,0.0,0.0,-99.0,1,0.0,0.0,1.0,0.0,...,0.095238,21,2,1,1,1.000000,1,0,0,1
11372,0,0.0,0.0,0.0,-99.0,1,1.0,0.0,0.0,0.0,...,1.000000,20,3,32,21,10.666667,1,0,0,1
7438,0,0.0,0.0,0.0,-99.0,1,1.0,0.0,0.0,0.0,...,0.703704,108,0,0,0,0.000000,1,0,0,1
2898,0,0.0,0.0,0.0,-99.0,6,0.0,0.0,1.0,0.0,...,0.500000,10,2,7,6,3.500000,1,0,0,1
10748,1,1.0,0.0,0.0,0.0,1,1.0,0.0,0.0,0.0,...,0.000000,15,0,0,0,0.000000,1,0,1,0
6855,0,0.0,0.0,0.0,-99.0,1,1.0,0.0,0.0,0.0,...,0.666667,3,0,0,0,0.000000,1,0,1,0
4994,0,0.0,0.0,0.0,-99.0,1,1.0,0.0,0.0,0.0,...,0.583333,24,0,0,0,0.000000,1,0,0,1


In [15]:
scoring_uri = 'http://545791ed-d646-4411-8d0b-e9ad868a980c.westeurope.azurecontainer.io/score'

In [16]:
import requests
import json

scoring_dataset = scoring_sample.values.tolist()
# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": scoring_dataset})

# Set the content type
headers = { 'Content-Type':'application/json' }

results = json.loads(requests.post(scoring_uri , input_json, headers = headers).json())['data']

In [17]:
for i in range(0, len(scoring_dataset)):
    flag = ['NON RIAMMESSO' if results['predictions'][i] == 0 else 'RIAMMESSO'][0]
    prob = results['probabilities'][i]
    print('Paziente %s con probabilità %s' % (str(flag), str(prob)))

Paziente NON RIAMMESSO con probabilità 0.9922619047619047
Paziente NON RIAMMESSO con probabilità 0.9522298010113135
Paziente NON RIAMMESSO con probabilità 1.0
Paziente RIAMMESSO con probabilità 0.7179137127492391
Paziente RIAMMESSO con probabilità 0.8051282342773941
Paziente NON RIAMMESSO con probabilità 0.9992857142857143
Paziente NON RIAMMESSO con probabilità 0.660691326064225
Paziente NON RIAMMESSO con probabilità 1.0
Paziente NON RIAMMESSO con probabilità 0.9985714285714287
Paziente NON RIAMMESSO con probabilità 0.9889610389610388
